In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import io

In [6]:
splitdir = Path("/media/monaco/DATA1/case_study/24h_10mmMAX_OI/split")
convective = pd.read_csv(splitdir / "cluster_convective.csv", sep=";")
intermediate = pd.read_csv(splitdir / "cluster_intermediate.csv", sep=";")
stratiform = pd.read_csv(splitdir / "cluster_stratiform.csv", sep=";")

dates = pd.concat([
    convective.assign(NAME="convective"),
    intermediate.assign(NAME="intermediate"),
    stratiform.assign(NAME="stratiform")
])

In [7]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=9, random_state=42, shuffle=True)
n_split = 2
train_index, test_index = list(skf.split(dates, dates.NAME))[n_split]
val_index, train_index = np.split(train_index, [len(test_index)])

print(len(train_index), len(val_index), len(test_index))

314 45 45


# Result analysis

In [7]:
respath = Path("lightning_logs")
for model in respath.glob("[!_]*"):
    for split in model.glob("split_*"):
        for predpath in (split / "test_images").glob("_pred.png"):
            pred = Image.open(predpath).convert("L")

lightning_logs/unet/split_5
lightning_logs/unet/split_3
lightning_logs/ensemble_unet/split_1
lightning_logs/ensemble_unet/split_3
lightning_logs/sde_unet/split_5
lightning_logs/sde_unet/split_3


# Xtreme events

In [8]:
input_path = Path('/media/monaco/DATA1/case_study')
n_split = 8
case_study = '24h_10mmMAX_OI'
input_path = input_path / case_study
use_extreme = False

case_study_max, available_models, train_dates, val_dates, test_dates, indices_one, indices_zero, mask, nx, ny = io.get_casestudy_stuff(
    input_path, n_split, case_study=case_study, ispadded=True,
    seed=42
)

print(len(train_dates), len(val_dates), len(test_dates), sum(len(x) for x in [train_dates, val_dates, test_dates]))

from sklearn.model_selection import train_test_split
test_dates1 = pd.concat([pd.read_csv(exev, header=None) for exev in input_path.glob('*extremeEvents.csv')])[0].values
train_dates = np.array(list(set(train_dates).union(val_dates).union(test_dates).difference(test_dates1)))
train_dates, val_dates = train_test_split(train_dates, train_size=315, random_state=42)
test_dates = test_dates1
print(len(train_dates), len(val_dates), len(test_dates), sum(len(x) for x in [train_dates, val_dates, test_dates])-17)




x_train, y_train, in_features, out_features = io.load_data(input_path, train_dates, case_study_max, indices_one, indices_zero, available_models)
x_val, y_val, in_features, out_features = io.load_data(input_path, val_dates, case_study_max, indices_one, indices_zero, available_models)
x_test, y_test, in_features, out_features = io.load_data(input_path, test_dates, case_study_max, indices_one, indices_zero, available_models)

315 45 45 405
315 59 48 405


AssertionError: File /media/monaco/DATA1/case_study/24h_10mmMAX_OI/models/bol00_20220606_0024_regrid.csv does not exist

In [14]:
for n, sp in zip(['tr', 'vl', 'ts'], [train_dates, val_dates, test_dates]):
    print(n)
    for d in sp:
        if not Path(f"/media/monaco/DATA1/case_study/24h_10mmMAX_OI/models/bol00_{d}_0024_regrid.csv").exists():
            print('>>', d)

tr
vl
ts
>> 20220606
>> 20220701
>> 20220727
>> 20220809
>> 20220906
>> 20220909
>> 20220928
>> 20221001
>> 20221223
>> 20230413
>> 20230516
>> 20230703
>> 20230825
>> 20230912
>> 20231113
>> 20231114
>> 20240318
